In [4]:
import numpy as np
import cv2
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from deepface import DeepFace
import time

# Load your trained model
model = tf.keras.models.load_model('face_recognition_mlp_model.h5')

# Load label encoder classes with allow_pickle=True
label_encoder = LabelEncoder()
label_encoder.classes_ = np.load('classes.npy', allow_pickle=True)

# Function to preprocess the face for your model
def preprocess_face(face):
    face = cv2.resize(face, (160, 160))  # Ukuran input untuk Facenet biasanya 160x160
    embedding = DeepFace.represent(face, model_name='Facenet', enforce_detection=False, detector_backend='retinaface')
    face = np.array(embedding[0]['embedding'])  # Ambil vektor embedding
    face = np.expand_dims(face, axis=0)
    return face

def capture_and_recognize(frame):
    print("Tombol ditekan, melakukan pengenalan...")  # Pesan debugging
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces using Haar Cascades
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    if len(faces) == 0:
        print("Tidak ada wajah terdeteksi.")  # Pesan debugging
    else:
        for (x, y, w, h) in faces:
            print(f"Wajah terdeteksi di posisi: {x}, {y}, {w}, {h}")  # Pesan debugging
            # Extract the face
            face = frame[y:y+h, x:x+w]

            try:
                preprocessed_face = preprocess_face(face)
                print("Fitur wajah berhasil diekstraksi.")  # Pesan debugging

                # Predict the face using the model
                predictions = model.predict(preprocessed_face)
                name_index = np.argmax(predictions, axis=1)

                # Convert index to name
                name = label_encoder.inverse_transform(name_index)[0]
                print(f"Nama yang dikenali: {name}")  # Pesan debugging

                # Annotate frame with the name
                cv2.putText(frame, name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
            except Exception as e:
                print(f"Error processing face: {e}")

# Inisialisasi video capture
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Display the video
    cv2.imshow('Video', frame)

    # Detect keypress event
    key = cv2.waitKey(1) & 0xFF
    if key == ord('r'):  # Ketika tombol 'r' ditekan, lakukan pengambilan gambar dan prediksi
        capture_and_recognize(frame)
    elif key == ord('q'):  # Tekan 'q' untuk keluar
        break

    # Tambahkan sedikit waktu tunggu untuk mencegah beban berlebih pada CPU
    time.sleep(0.03)  # Sesuaikan sesuai kebutuhan

# Release video capture
cap.release()
cv2.destroyAllWindows()


Tombol ditekan, melakukan pengenalan...
Wajah terdeteksi di posisi: 246, 75, 288, 288
Fitur wajah berhasil diekstraksi.
1/1 [==============================] - 0s 55ms/step
Nama yang dikenali: Fadh
Tombol ditekan, melakukan pengenalan...
Wajah terdeteksi di posisi: 347, 132, 128, 128
Fitur wajah berhasil diekstraksi.
1/1 [==============================] - 0s 20ms/step
Nama yang dikenali: Abira Azahra
Tombol ditekan, melakukan pengenalan...
Wajah terdeteksi di posisi: 315, 136, 130, 130
Fitur wajah berhasil diekstraksi.
1/1 [==============================] - 0s 21ms/step
Nama yang dikenali: Abira Azahra
Tombol ditekan, melakukan pengenalan...
Wajah terdeteksi di posisi: 313, 118, 172, 172
Fitur wajah berhasil diekstraksi.
1/1 [==============================] - 0s 20ms/step
Nama yang dikenali: Fadh
Tombol ditekan, melakukan pengenalan...
Wajah terdeteksi di posisi: 276, 98, 299, 299
Fitur wajah berhasil diekstraksi.
1/1 [==============================] - 0s 18ms/step
Nama yang dikenali: F